# Basic plotting tools

hv resources
- Gallery: https://holoviews.org/reference/
- https://holoviews.org/user_guide/Customizing_Plots.html
- https://holoviews.org/user_guide/Style_Mapping.html
- https://holoviews.org/user_guide/Colormaps.html
- https://holoviews.org/user_guide/Plotting_with_Bokeh.html

In [ ]:
import holoviews as hv

import hv_anndata
from hv_anndata import ACCESSOR as A
from hv_anndata import register

register()

hv.extension("bokeh")

In [ ]:
import numpy as np
import scanpy as sc

rng = np.random.default_rng()

In [ ]:
adata = sc.datasets.pbmc68k_reduced()
adata.layers["counts"] = adata.raw.X
del adata.raw
sc.tl.umap(adata)
adata

[pl.scatter](https://scanpy.readthedocs.io/en/stable/generated/scanpy.pl.scatter)

missing features:
- `use_raw` (deprecated!)
- `na_color` (not super important)

missing convenience:
- `basis` for easy X&Y

In [ ]:
# add NAs to check how missing values look
adata_scatter = adata.copy()
adata_scatter.obs.loc[
    (
        (adata_scatter.obs["bulk_labels"] == "Dendritic")
        & rng.choice([True, False], size=len(adata_scatter))
    ),
    "bulk_labels",
] = np.nan

hv.Scatter(
    adata_scatter, A.obsm["X_umap"][0], [A.obsm["X_umap"][1], A.obs["bulk_labels"]]
).opts(
    color=A.obs["bulk_labels"], cmap="tab10", aspect="square", legend_position="right"
)

[pl.heatmap](https://scanpy.readthedocs.io/en/stable/generated/scanpy.pl.heatmap.html)

missing:
- `groupby` / [TickBar](https://github.com/holoviz/holoviews/issues/6658)
- `standard_scale` (see implemantation in {class}`hv_anndata.Dotmap`)

In [ ]:
markers = ["C1QA", "PSAP", "CD79A", "CD79B", "CST3", "LYZ"]
hv.HeatMap(
    adata[np.argsort(adata.obs["bulk_labels"], stable=True), markers],
    [A.obs["index"], A.var["index"]],
    [A[:, :]],
).opts(xticks=0, colorbar=True, responsive=True, height=200)

[pl.dotplot](https://scanpy.readthedocs.io/en/stable/generated/scanpy.pl.dotplot.html)

missing:
- `var_group_*` (highlight groups of `var_names` by drawing brackets)

In [ ]:
hv_anndata.Dotmap(adata=adata, marker_genes=markers, groupby="bulk_labels")

[pl.tracksplot](https://scanpy.readthedocs.io/en/stable/generated/scanpy.pl.tracksplot.html)

missing:
- `shared_xaxis=False` doesn’t seem to work on GridSpace
- GridSpace can only be 2D or row-only, not col-only
- how to display shared legend?

In [ ]:
markers = ["C1QA", "PSAP", "CD79A", "CD79B", "CST3", "LYZ"]
hv.NdLayout(
    {
        m: hv.Curve(adata, [A.obs["index"]], [A[:, m], A.obs["bulk_labels"]])
        .opts(
            xticks=0,
            xlabel="",
            ylabel=m,
            title="",
            frame_height=50,
            frame_width=900,
            show_legend=False,
        )
        .groupby(A.obs["bulk_labels"], hv.NdOverlay)
        for m in markers
    },
    kdims=["marker"],
).cols(1)

In [ ]:
markers = ["C1QA", "PSAP", "CD79A", "CD79B", "CST3", "LYZ"]
hv.GridSpace(
    {
        (0, m): hv.Curve(adata, [A.obs["index"]], [A[:, m], A.obs["bulk_labels"]])
        .opts(frame_height=50, frame_width=900)
        .groupby(A.obs["bulk_labels"], hv.NdOverlay)
        for m in markers
    },
    kdims=["_", "marker"],
)

pl.violin

pl.stacked_violin

pl.matrixplot

pl.clustermap

pl.ranking

pl.dendrogram